In [1]:
using Plots #v0.29.9
using DataFrames #v0.22.5
using IterableTables #v1.0.0
using LsqFit #v0.10.0
using Dierckx #v0.4.2
using DifferentialEquations #v6.10.1
using ColorSchemes #v3.10.2
using Statistics
using CSV
theme(:wong)
pyplot()

Plots.PyPlotBackend()

In [2]:
#Define strain object
mutable struct Strain 
    Vm::Float64 #Vmax
    Km::Float64 #Km
    
    TEb::Float64 #basal transcription rate for the genes belonging to the E fraction
    TRb::Float64 #basal transcription rate for the genes belonging to the R fraction
    TsE::Float64 #maximum transcription rate for the genes of thE E fraction
    TsR::Float64 #maximum transcription rate for the genes of thE R fraction
    
    n::Float64 #Hill coeficient
    θE::Float64 #activity threshold for ppGpp
    θR::Float64 #activity threshold for ppGpp
    θG::Float64 #synthesis threshold for ppGpp
    
    TlE::Float64 #maxumim translation rate for the mRNA of the E fraction
    TlR::Float64 #maxumim translation rate for the mRNA of the R fraction
    
    ηE::Float64 #degradation rate of the trascripts belonging to the E sector
    ηR::Float64 #degradation rate of the trascripts belonging to the R sector
    δE::Float64 #degradation rate of molecules belonging to the E sector
    δR::Float64 #degradation rate of proteins belonging to the R sector
    δG::Float64 #degradation rate of ppGpp
    
    JE::Float64 #fraction of ribosomes translating proteins belonging to the E sector
    JR::Float64 #fraction of ribosomes translating proteins belonging to the R sector
    
    σb::Float64 #basal ppGpp synthesis rate
    σi::Float64 #induced ppGpp synthesis rate

    kn::Float64 #Proportionalilty constant for Growth rate ~ Ribosomes
    c::Float64 #resourse conversion constant
    
    leak::Float64 #Leakage coefficient
    ψ::Float64 #Amino acid cost
end

# Proteome model

## Equations

In [3]:
function model_proteome(du,u,p,t)
    mE, mR, e, aap, R, G = u
    Vm, S, aa, Km, TEb, TsE, n, θE, ηE, TRb, TsR, θR, ηR, ψ, TlE, JE, δE, TlR, JR, δR, σb, σi, θG, δG = p
    
    #Transcription
    du[1] = dmE = (((Vm*(S*aa)) / (Km+(S*aa))) * TEb) + TsE * (G^n/(G^n+θE^n)) - (ηE * mE)
    du[2] = dmR = (((Vm*(S*aa)) / (Km+(S*aa))) * TRb) + TsR * (θR^n/(G^n+θR^n)) - (ηR * mR)
    
    #Proteome fractions
    du[3] = de = (1-ψ) * (TlE * mE) * (1-e/R*JE) - (δE * e)
    du[4] = daap = ψ * (TlE * mE) * (1-aap/R*JE) - (δE * aap)
    du[5] = dR = (TlR * mR) * (1-R/R*JR) - (δR * R)
    
    #Resource allocation regulation
    du[6] = dG = σb + (σi * (R^n / (R^n + θG^n))) - (δG * G) #Strain

end

model_proteome (generic function with 1 method)

## Parameters

In [4]:
mE0 = 1.0 #E fraction mRNA concentration
mR0 = 1.0 #E fraction mRNA concentration
e0 = 1.0 #Catabolic sector of the E fraction
aap0 = 1.0 #Amino acid production sector of the E fraction
R0 = 1.0 #R fraction
G0 = 0.01 #ppGpp

u0_prot = [mE0; mR0; e0; aap0; R0; G0]

#Time interval
tspan_prot = (0.0,50.0)

(0.0, 50.0)

In [5]:
#Strain X
stx = Strain(15.0, #Vmax
    1.0, #Km
    0.5, #TEb
    0.5, #TRb
    1.0, #TsE
    1.0, #TsR
    2.0, #n
    1.0, #θE
    1.0, #θR
    1.0, #θG
    1.0, #TlE
    1.0, #TlR
    1.0, #ηE
    1.0, #ηR
    0.01, #δE
    0.01, #δR
    0.01, #δG
    0.5, #JE
    0.5, #JR
    0.1, #σb
    0.5, #σi
    2.0, #kn
    1, #e8, #c
    0.5, #leak
    0.5 #ψ
)

Strain(15.0, 1.0, 0.5, 0.5, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.01, 0.01, 0.01, 0.5, 0.5, 0.1, 0.5, 2.0, 1.0, 0.5, 0.5)

In [6]:
#Strain Y
#I know they're exactly the same, but julia gets confused if I don't do it this way

sty = Strain(15.0, #Vmax
    1.0, #Km
    0.5, #TEb
    0.5, #TRb
    1.0, #TsE
    1.0, #TsR
    2.0, #n
    1.0, #θE
    1.0, #θR
    1.0, #θG
    1.0, #TlE
    1.0, #TlR
    1.0, #ηE
    1.0, #ηR
    0.1, #δE
    0.1, #δR
    0.1, #δG
    0.5, #JE
    0.5, #JR
    0.1, #σb
    0.5, #σi
    1.0, #kn
    1, #e8, #c
    0.5, #leak
    0.5 #ψ
)

Strain(15.0, 1.0, 0.5, 0.5, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.1, 0.1, 0.1, 0.5, 0.5, 0.1, 0.5, 1.0, 1.0, 0.5, 0.5)

## Figure 1D & 1E

In [7]:
function run_simulation(st)
    df = DataFrame(lambda = Float64[], betas = Float64[])
    s_s = []
    aa_s = []
    
   for aa = collect(0.0:0.1:10.0)
        for S = collect(0.0:0.1:10.0)

            p = [st.Vm, S, aa, st.Km, st.TEb, st.TsE, st.n, st.θE, st.ηE, st.TRb, st.TsR, st.θR, st.ηR, st.ψ, st.TlE, st.JE, st.δE, st.TlR, st.JR, st.δR, 
                    st.σb, st.σi, st.θG, st.δG]
            prob = ODEProblem(model_proteome,u0_prot,tspan_prot,p)
            sol = solve(prob,reltol=1e-8,abstol=1e-8,saveat=0.1, isoutofdomain=(u0_prot,p,tspan) -> any(x -> x < 0, u0_prot))
            push!(s_s, S)
            push!(aa_s, aa)

            #This step determines the final growth rate and proportion of E sector producing aminoacid
            push!(df, [sol[5,end] * st.kn, sol[4,end] * st.leak])     
        end
    end
    
    values = DataFrame(S=s_s, aa = aa_s)
    results = hcat(values, df)  
    plot(results[:,:S], results[:,:aa], results[:,:lambda], st=:surface, camera=(-60,30), xaxis="Glucose (S)", yaxis="Amino acid (aa)", zaxis="Growth rate (λ)")
    fitting(results)
    
end

function get_surface(st)
    df = DataFrame(lambda = Float64[], betas = Float64[])
    s_s = []
    aa_s = []
    
    step=0.1
    aa_max=10
    S_max=10
   for aa = collect(0.0:step:S_max)
        for S = collect(0.0:step:aa_max)

            p = [st.Vm, S, aa, st.Km, st.TEb, st.TsE, st.n, st.θE, st.ηE, st.TRb, st.TsR, st.θR, st.ηR, st.ψ, st.TlE, st.JE, st.δE, st.TlR, st.JR, st.δR, 
                    st.σb, st.σi, st.θG, st.δG]
            prob = ODEProblem(model_proteome,u0_prot,tspan_prot,p)
            sol = solve(prob,reltol=1e-8,abstol=1e-8,saveat=0.1, isoutofdomain=(u0_prot,p,tspan) -> any(x -> x < 0, u0_prot))
            
            push!(s_s, S)
            push!(aa_s, aa)
            
            #This step determines the final growth rate and proportion of E sector producing aminoacid
            push!(df, [sol[5,end] * st.kn, sol[4,end] * st.leak])     
        end
    end
    
    values = DataFrame(S=s_s, aa = aa_s)
    results = hcat(values, df)  
    sp_aa, sp_beta=get_surface(results)
    
    return sp_aa, sp_beta
end


get_surface (generic function with 1 method)

In [8]:
function plot_proteome_run(st)
    
    plt1 = plot(xaxis="Time (t)", yaxis="mRNA concentration",  xtickfontsize=14, ytickfontsize=14, xguidefontsize=16, yguidefontsize=16, legend = :outerright, 
        legendfontsize=12)
    plot!(size=(480,320))
    
    plt2 = plot(xaxis="Time (t)", yaxis="Protein concentration", title="", xtickfontsize=14, ytickfontsize=14, xguidefontsize=16, yguidefontsize=16, legend = :outerright, 
        legendfontsize=12)
    plot!(size=(480,320))
    
    plt3 = plot(xaxis="Time (t)", yaxis="Proteome fraction (%)", title="", xtickfontsize=14, ytickfontsize=14, xguidefontsize=16, yguidefontsize=16, legendfontsize=12, legend = :outerright)
    plot!(size=(480,320))
    
    ln_type = [:solid, :dot]
    nutrient = [10, 0.5] 
    index2 = [1:2, 3:4]
    index3 = [1:4, 5:8]
        
    labels_1 = ["mE (rich)" "mR " "mE (poor)" "mR "]
    labels_2 = ["ϵ (rich)" "α " "R " "γ " "ϵ (poor)" "α " "R " "γ "]
    labels_3 = ["E (rich)" "R " "E (poor)" "R "]
    labels_4 = ["ϵ" "α" "R" "G"]
    
    col_p = ["#59A96A" "#6A1148" "#006D77" "#DB8B00"]
    v_p = [3;4;5;6]

    #Plot for rich and low substrate
    for i = 1:2
        aa = nutrient[i] #
        S= nutrient[i]
        #aa = nutrient[i]

        p = [st.Vm, S, aa, st.Km, st.TEb, st.TsE, st.n, st.θE, st.ηE, st.TRb, st.TsR, st.θR, st.ηR, st.ψ, st.TlE, st.JE, st.δE, st.TlR, st.JR, st.δR, st.σb, st.σi, 
            st.θG, st.δG]
        prob = ODEProblem(model_proteome,u0_prot,tspan_prot,p)
        sol = solve(prob,reltol=1e-8,abstol=1e-8,saveat=0.1, isoutofdomain=(u0_pop,p,tspan) -> any(x -> x < 0, u0_prot))

        E = sol[3,:] .+ sol[4,:]
        R = sol[5,:]
        Z = E .+ R
        tot = E .+ R .+ Z
        prot = [E.* 100 ./ tot, R .* 100 ./ tot]
        labels = ["E", "R"]

        plot!(plt1, sol, vars=1:2, color=["#59A96A" "#58BAEE"], line = ln_type[i], lab=permutedims(labels_1[index2[i]]), lw = 3,xaxis="Time (t)", yaxis="mRNA concentration", legend = :outerright, grid=false)
        plot!(plt2, sol, vars=v_p, color=col_p, line = ln_type[i], lab=permutedims(labels_2[index3[i]]), lw = 3,xaxis="Time (t)", yaxis="Protein concentration", legend = :outerright, grid=false)
        plot!(plt3, sol.t, prot, color=["#59A96A" "#006D77"], line = ln_type[i], lab=permutedims(labels_3[index2[i]]), lw = 3,xaxis="Time (t)", yaxis="Proteome fraction (%)", legend = :outerright, grid=false, ylim=(10,40))

    end
    
    
    display(plt1) 
    display(plt2) 
    display(plt3)
end

plot_proteome_run (generic function with 1 method)

In [9]:
function fitting(results)
    spl_Vm = Spline2D(results[:, :aa], results[:, :S], results[:, :lambda], s=1000000) #Fit Vmax
    spl_beta = Spline2D(results[:, :aa], results[:, :S], results[:, :betas], s=1000000) #Fit beta
    return spl_Vm, spl_beta
end

fitting (generic function with 1 method)

In [10]:
function get_params(spl_Vm, spl_beta, aa, S)
    
    Vmax_fit = spl_Vm(S, aa) #Interpolates Vmax
    Beta_fit = spl_beta(S, aa) #Interpolates Beta
    
    return [Vmax_fit; Beta_fit]
end

get_params (generic function with 1 method)

In [11]:
#plot_proteome_run(stx)

# Population Dynamics Model

## Parameters

In [12]:
S0 = 0.1 #Initial resource concentration
X0 = 0.1 #Initial amino acid X concentration
Y0 = 0.1 #Initial amino acid Y concentration
D = 0.0 #Dilution rate

tspan = (0.0,0.1) #Time interval

(0.0, 0.1)

In [13]:

function model_population(du,u,p,t)
    #X0,Y0=0.0 #tmp
    S0 = u[1]
    S, X, Y, Bx, By = u
    Vmx, Kmx, Vmy, Kmy, βx, βy, cx, cy = p
    
    du[1] = dS = - (Us(S,Vmx,Kmx) * Bx) - (Us(S,Vmy,Kmy) * By)  - D*(S-S0) #Substrate
    du[2] = dX = .0001*βx * G(S,cx,Vmx,Kmx,Y) * By - (Uaa(X) * Bx) - D*X #X
    du[3] = dY = .0001*βy * G(S,cy,Vmy,Kmy,X) * Bx - (Uaa(Y) * By) - D*Y #Y
    du[4] = dBx = G(S,cx,Vmx,Kmx,X) * Bx - D*Bx #Bx
    du[5] = dBy = G(S,cy,Vmy,Kmy,Y) * By - D*By #Bx

end

function Us(S, Vmax, Km)
    Us = (Vmax * S) / (Km + S)
end

function Uaa(aa) #
    Uaa = aa/(aa+1)
end

function G(S,c, Vmax, Km, aa)  #Resource consumption
    G = c * Us(S, Vmax, Km) * Uaa(aa)
end


G (generic function with 1 method)

In [14]:
#Run the populations model and fit the Vmax and Beta parameters
function simulate(T, u0, spl_Vmx, spl_betax, spl_Vmy, spl_betay, stx, sty)    
      
    ncycles=T
    
    u0_pop = [u0[1], u0[2], u0[3], u0[4], u0[5]]
    
    results = DataFrame(S = Float64[], aa_x = Float64[], aa_y = Float64[], Bx = Float64[], By = Float64[], Vmx = Float64[], Vmy = Float64[], Betax = Float64[], 
        Betay = Float64[])
    
    fit_x0 = get_params(spl_Vmx, spl_betax, u0_pop[2], u0_pop[1])
    fit_y0 = get_params(spl_Vmy, spl_betay, u0_pop[3], u0_pop[1])

    push!(results, [u0_pop[1]; u0_pop[2]; u0_pop[3]; u0_pop[4]; u0_pop[5]; fit_x0[1]; fit_y0[1]; fit_x0[2]; fit_y0[2]])
    
    
    for i in 0:ncycles-1
        tspan_cycle =[i*ncycles, (i+1)*ncycles]  #Time interval
        
        fit_x = get_params(spl_Vmx, spl_betax, u0_pop[2], u0_pop[1])
        fit_y = get_params(spl_Vmy, spl_betay, u0_pop[3], u0_pop[1])
        
        #p = [Vmx, Kmx, Vmy, Kmy, cx, cy, Bx, By]
        p = [fit_x[1], 1.0, fit_y[1], 1.0, fit_x[2], fit_y[2], stx.c, sty.c]
        
        pop = ODEProblem(model_population, u0_pop, tspan_cycle, p)
        sol_pop = solve(pop, Rosenbrock23(autodiff=false), maxiters = 1e7, reltol=1e-10, abstol=1e-10, isoutofdomain=(u0_pop,p,tspan) -> any(x -> x < 0, u0_pop)) #, saveat=0.1
        
        push!(results, [sol_pop[:,end]; fit_x[1]; fit_y[1]; fit_x[2]; fit_y[2]])
        
        #nsteps=10
        #ts = collect(tspan[1]:(tspan[2]-tspan[1])/nsteps:tspan[2])
        #for i in 2:nsteps
        #    push!(results, [sol_pop(ts[i]); fit_x[1]; fit_y[1]; fit_x[2]; fit_y[2]])
        #end
        
        #u0_pop = [sol_pop[1,end]; sol_pop[2,end]; sol_pop[3,end]; sol_pop[4,end]; sol_pop[5,end]]
        u0_pop = [sol_pop[1,end] + D * (S0 - sol_pop[1,end]); sol_pop[2,end]; sol_pop[3,end]; sol_pop[4,end] - (D * sol_pop[4,end]); sol_pop[5,end] - (D * sol_pop[5,end])]
    end
    results   
end

simulate (generic function with 1 method)

In [15]:
function plotRun(res)#, parx, pary, title)
     
    plt_title = plot(title = "Run", grid = false, showaxis = false, bottom_margin = -100Plots.px)

    p1 = plot(res[:,:S], lab="S", xaxis="", yaxis="Concentration (μg/mL)", linecolor = "#883677", lw = 2)#, title = "A", bottom_margin = 10Plots.px, 
        #titleloc = :left)
    plot!(res[:,:aa_x], lab="X", linecolor = "#084C61", lw = 2)
    plot!(res[:,:aa_y], lab="Y", linecolor = "#DB3A34", lw = 2)
    
    Btot=maximum(res[:,:Bx]+res[:,:By])
    p2 = plot(res[:,:Bx], lab="Bx", xaxis="", yaxis="Density (cells/mL)", linecolor = "#177E89", lw = 2, ylims = (0,1.1*Btot))#, ylims = (0,1.1*Btot))#, title = "B", bottom_margin = 10Plots.px, 
        #titleloc = :left)
    plot!(res[:,:By], lab="By", linecolor = "#FFC857", lw = 2)
    
    Vmax=maximum([maximum(res[:,:Vmx]),maximum(res[:,:Vmy])])
    p3 = plot(res[:,:Vmx], lab="X", xaxis="Time (t)", yaxis="λ", ylims = (0,1.1*Vmax), linecolor = "#491D40", lw = 2)#, title = "C",  bottom_margin = 10Plots.px, 
        #titleloc = :left)
    plot!(res[:,:Vmy], lab="Y", linecolor = "#C56DB3", lw = 2)
    #plot!(size=(300,200))
    
    Bmax=maximum([maximum(res[:,:Betax]),maximum(res[:,:Betay])])
    p4 = plot(res[:,:Betax], lab="X", xaxis="Time (t)", yaxis="β", ylims = (0,1.1*Bmax), linecolor = "#2ECADC", lw = 2)#, title = "D", bottom_margin = 10Plots.px, 
        #titleloc = :left)
    plot!(res[:,:Betay], lab="Y", linecolor = "#FFB41F", lw = 2)
    #plot!(size=(300,200))
    
    
    #p=plot(p1, p2, p3, p4, layout = @layout([B C]; [D E]))
    p=plot(plt_title, p1, p2, p3, p4, layout = @layout([A{0.3h}; [B C]; [D E]]))

    return p
    
end

plotRun (generic function with 1 method)

## Run simulations
### Plot time lapse

In [ ]:
u0_pop = [1; X0; Y0; 1e6; 1e6]

spl_aax, spl_betax = run_simulation(stx)
spl_aay, spl_betay = run_simulation(sty)

T=3
res = simulate(T, u0_pop, spl_aax, spl_betax, spl_aay, spl_betay, stx, sty)

print(res)

#plt=plotRun(res)#, stx.θG, sty.θG, "θG")
    #plot!(size=(400,300))

In [ ]:
plt=plotRun(res)#, stx.θG, sty.θG, "θG")
#display(plt)
#savefig("../figures/src/Figure2B.png")